Copyright 2022 Google LLC. This software is provided as-is, without warranty or representation for any use or purpose. Your use of it is subject to your agreements with Google.

In [ ]:
import os
from google.cloud import storage
from google.cloud import bigquery

In [ ]:
region = "us-central1"

bucket_name = "uhc-json-test" #GCS Bucket name where the CMS JSON resides
prefix = "test/" #Path under the bucket to the file with a trailing / - if it's in the root, just ""
filename = "2022-07-01_UnitedHealthcare-Insurance-Company-of-New-York_Insurer_Behavior-Health_P3_in-network-rates.json.gz" #Filename of the gzipped json
json_file = os.path.splitext(filename)[0]

bq_dataset = "mydataset" #BigQuery dataset 
bq_in_network_table = "UHC_p3_in_network" #Temporary in-network table name
bq_remaining_table = "UHC_p3" #Temporary table for everything in the JSON except the in-network table
merged_table = "uhc_p3_merged" #Name of the merged table


In [ ]:
storage_client = storage.Client()

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(prefix+filename)
blob.download_to_filename(filename)

In [ ]:
!gunzip $filename

In [ ]:
!jq -c '."in_network"[]' $json_file > {bq_in_network_table}.json

In [ ]:
!jq -c 'del(."in_network")' $json_file > {bq_remaining_table}.json

In [ ]:
!bq --location={region} load  --autodetect --source_format=NEWLINE_DELIMITED_JSON {bq_dataset+"."+bq_in_network_table} {bq_in_network_table}.json

In [ ]:
!bq --location={region} load  --autodetect --source_format=NEWLINE_DELIMITED_JSON {bq_dataset+"."+bq_remaining_table} {bq_remaining_table}.json

In [ ]:
client = bigquery.Client()

query = (
    """CREATE TABLE {}.{} AS SELECT * FROM 
    (SELECT * FROM {}.{}),
    (SELECT * FROM {}.{})""").format(bq_dataset,merged_table,bq_dataset,bq_remaining_table,bq_dataset,bq_in_network_table)

query_job = client.query(query)
result = query_job.result()

print(result)

In [ ]:
#Optional - delete the intermediate tables
client.delete_table(bq_dataset+"."+bq_in_network_table, not_found_ok=True)
print("Deleted table '{}'.".format(bq_in_network_table))

client.delete_table(bq_dataset+"."+bq_remaining_table, not_found_ok=True)
print("Deleted table '{}'.".format(bq_remaining_table))